In [0]:
import numpy as np
import pandas as pd
import datetime
import random

# Dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!unzip /content/drive/My\ Drive/dot_traffic_2015.txt.gz.zip

Archive:  /content/drive/My Drive/dot_traffic_2015.txt.gz.zip
  inflating: dot_traffic_2015.txt.gz  


In [0]:
traffic_df = pd.read_csv('/content/dot_traffic_2015.txt.gz', compression='gzip', header=0, sep=',', quotechar='"')

In [0]:
del traffic_df['functional_classification_name']
del traffic_df['date']
del traffic_df['restrictions']
del traffic_df['year_of_data']
del traffic_df['record_type']
del traffic_df['direction_of_travel_name']
del traffic_df['fips_state_code']
del traffic_df['station_id']

In [33]:
traffic_df.head(10)

,day_of_data,day_of_week,direction_of_travel,functional_classification,lane_of_travel,month_of_data,traffic_volume_counted_after_0000_to_0100,traffic_volume_counted_after_0100_to_0200,traffic_volume_counted_after_0200_to_0300,traffic_volume_counted_after_0300_to_0400,traffic_volume_counted_after_0400_to_0500,traffic_volume_counted_after_0500_to_0600,traffic_volume_counted_after_0600_to_0700,traffic_volume_counted_after_0700_to_0800,traffic_volume_counted_after_0800_to_0900,traffic_volume_counted_after_0900_to_1000,traffic_volume_counted_after_1000_to_1100,traffic_volume_counted_after_1100_to_1200,traffic_volume_counted_after_1200_to_1300,traffic_volume_counted_after_1300_to_1400,traffic_volume_counted_after_1400_to_1500,traffic_volume_counted_after_1500_to_1600,traffic_volume_counted_after_1600_to_1700,traffic_volume_counted_after_1700_to_1800,traffic_volume_counted_after_1800_to_1900,traffic_volume_counted_after_1900_to_2000,traffic_volume_counted_after_2000_to_2100,traffic_volume_counted_after_2100_to_2200,traffic_volume_counted_after_2200_to_2300,traffic_volume_counted_after_2300_to_2400
0,7,3,1,3R,1,4,4,3,2,4,43,78,116,144,132,115,150,184,169,136,129,89,122,124,110,69,73,28,12,6
1,26,7,7,1U,2,9,381,252,218,194,220,348,453,679,826,962,1158,1379,1376,1383,1453,1617,1669,1308,1068,928,885,798,650,613
2,16,3,3,1U,0,6,585,408,328,364,696,1929,4228,5634,5673,4636,3925,3827,4049,3954,4077,4244,4405,4609,4361,3272,2243,2050,1453,892
3,26,1,5,1U,1,4,105,73,68,66,77,113,254,367,487,668,870,996,1003,1000,1043,1011,959,851,708,559,457,297,207,110
4,23,7,3,4R,0,5,6,4,2,3,1,10,17,52,64,68,82,96,99,87,87,83,61,55,35,29,21,23,9,7
5,25,7,5,2U,0,7,1168,781,738,425,279,395,715,1202,1486,1819,2193,2401,2691,2508,2700,2673,2746,2564,2257,2140,2060,2156,1873,1590
6,10,5,3,3U,2,9,34,43,27,48,88,269,334,476,444,355,375,434,492,499,539,542,571,559,395,351,262,217,153,61
7,27,3,3,4U,1,10,13,3,8,3,17,47,95,158,151,151,141,184,193,219,217,238,278,250,154,130,84,56,21,14
8,26,6,7,1R,2,6,115,78,31,40,42,134,268,335,392,441,600,722,652,706,751,740,784,822,703,425,192,159,150,168
9,12,3,5,2U,2,5,189,100,115,108,219,527,1212,1596,1347,1115,1114,1126,1178,1193,1364,1662,1673,1519,1148,914,808,555,468,270


In [6]:
traffic_df.shape

(7140391, 30)

In [34]:
one_hot_column = ['direction_of_travel', 'functional_classification','direction_of_travel','lane_of_travel','month_of_data','day_of_week','day_of_data']
prefix = []
for col in one_hot_column:
  prefix.append(col.replace('_',''))
data = pd.get_dummies(traffic_df,prefix=prefix, columns=one_hot_column)
data = data[:30000]
data[:5]

,traffic_volume_counted_after_0000_to_0100,traffic_volume_counted_after_0100_to_0200,traffic_volume_counted_after_0200_to_0300,traffic_volume_counted_after_0300_to_0400,traffic_volume_counted_after_0400_to_0500,traffic_volume_counted_after_0500_to_0600,traffic_volume_counted_after_0600_to_0700,traffic_volume_counted_after_0700_to_0800,traffic_volume_counted_after_0800_to_0900,traffic_volume_counted_after_0900_to_1000,traffic_volume_counted_after_1000_to_1100,traffic_volume_counted_after_1100_to_1200,traffic_volume_counted_after_1200_to_1300,traffic_volume_counted_after_1300_to_1400,traffic_volume_counted_after_1400_to_1500,traffic_volume_counted_after_1500_to_1600,traffic_volume_counted_after_1600_to_1700,traffic_volume_counted_after_1700_to_1800,traffic_volume_counted_after_1800_to_1900,traffic_volume_counted_after_1900_to_2000,traffic_volume_counted_after_2000_to_2100,traffic_volume_counted_after_2100_to_2200,traffic_volume_counted_after_2200_to_2300,traffic_volume_counted_after_2300_to_2400,directionoftravel_0,directionoftravel_1,directionoftravel_2,directionoftravel_3,directionoftravel_4,directionoftravel_5,directionoftravel_6,directionoftravel_7,directionoftravel_8,directionoftravel_9,functionalclassification_1R,functionalclassification_1U,functionalclassification_2U,functionalclassification_3R,functionalclassification_3U,functionalclassification_4R,...,monthofdata_11,monthofdata_12,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,dayofweek_7,dayofdata_1,dayofdata_2,dayofdata_3,dayofdata_4,dayofdata_5,dayofdata_6,dayofdata_7,dayofdata_8,dayofdata_9,dayofdata_10,dayofdata_11,dayofdata_12,dayofdata_13,dayofdata_14,dayofdata_15,dayofdata_16,dayofdata_17,dayofdata_18,dayofdata_19,dayofdata_20,dayofdata_21,dayofdata_22,dayofdata_23,dayofdata_24,dayofdata_25,dayofdata_26,dayofdata_27,dayofdata_28,dayofdata_29,dayofdata_30,dayofdata_31
0,4,3,2,4,43,78,116,144,132,115,150,184,169,136,129,89,122,124,110,69,73,28,12,6,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,381,252,218,194,220,348,453,679,826,962,1158,1379,1376,1383,1453,1617,1669,1308,1068,928,885,798,650,613,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,585,408,328,364,696,1929,4228,5634,5673,4636,3925,3827,4049,3954,4077,4244,4405,4609,4361,3272,2243,2050,1453,892,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,105,73,68,66,77,113,254,367,487,668,870,996,1003,1000,1043,1011,959,851,708,559,457,297,207,110,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,6,4,2,3,1,10,17,52,64,68,82,96,99,87,87,83,61,55,35,29,21,23,9,7,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [16]:
data.shape

(30000, 116)

In [0]:
cat_index = {} # Mapping of feature -> start index of feature in a record
cat_values = {} # Mapping of feature -> list of categorical values the feature can take
# build up the cat_index and cat_values dictionary
one_hot_column = ['direction_of_travel', 'functional_classification','direction_of_travel','lane_of_travel','month_of_data','day_of_week','day_of_data']
for i, header in enumerate(data.keys()):
  if i > 23: # categorical header
    feature, value = header.split('_')
    if feature not in cat_index:
      cat_index[feature] = i
      cat_values[feature] = [value]
    else:
      cat_values[feature].append(value)
def get_onehot(record, feature):
  """
  Return the portion of `record` that is the one-hot encoding
  of feature. For example, since the feature "work" is stored
  in the indices [5:12] in each record, calling `get_range(record, "work")`
  is equivalent to accessing `record[5:12]`.
  Args:
  - record: a numpy array representing one record, formatted
  the same way as a row in `data.np`
  - feature: a string, should be an element of `catcols`
  """
  start_index = cat_index[feature]
  stop_index = cat_index[feature] + len(cat_values[feature])
  return record[start_index:stop_index]
def get_categorical_value(onehot, feature):
  """
  Return the categorical value name of a feature given
  a one-hot vector representing the feature.
  Args:
  - onehot: a numpy array one-hot representation of the feature
  - feature: a string, should be an element of `catcols`
  Examples:
  >>> get_categorical_value(np.array([0., 0., 0., 0., 0., 1., 0.]), "work")
  'State-gov'
  >>> get_categorical_value(np.array([0.1, 0., 1.1, 0.2, 0., 1., 0.]), "work")
  'Private'
  """
  # TODO
  idx = np.argmax(onehot)
  return cat_values[feature][idx]
def get_feature(record, feature):
  """
  Return the categorical feature value of a record
  """
  onehot = get_onehot(record, feature)
  return get_categorical_value(onehot, feature)
def get_features(record):
  return { f: get_feature(record, f) for f in catcols }

In [10]:
get_categorical_value(np.array([0.1, 0., 1.1, 3.3, 0., 1., 0.,0,0,0,0,0,0,0,55,0,0,0]), "dayofdata")

'15'

#DataLoader

In [11]:
np.random.seed(50) # set the numpy seed for consistent split
arr = np.arange(len(data))
np.random.shuffle(arr)
np_data=data.values
shuffled_data=np_data
total_data=len(np_data)
train_data=shuffled_data[:int(total_data*0.7)].astype(np.float32)
validation_data=shuffled_data[int(total_data*0.7):int(total_data*0.85)].astype(np.float32)
test_data=shuffled_data[int(total_data*0.85):].astype(np.float32)
print("Training dataset size: {}".format(train_data.shape[0]))
print("Validation dataset size: {}".format(validation_data.shape[0]))
print("Test dataset size: {}".format(test_data.shape[0]))

Training dataset size: 21000
Validation dataset size: 4500
Test dataset size: 4500


#Neural Network

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim

In [0]:
cuda = torch.device('cuda')

In [0]:
def get_model_name(name, batch_size, learning_rate, epoch, momentum):
  """ Generate a name for the model consisting of all the hyperparameter values
  Args:
  config: Configuration object containing the hyperparameters
  Returns:
  path: A string with the hyperparameter name and value concatenated
  """
  if momentum:
    path = "model_{0}_bs{1}_lr{2}_epoch{3}_momentum{4}".format(name,
            batch_size,
            learning_rate,
            epoch,momentum)
  else:
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
            batch_size,
            learning_rate,
            epoch)
  return path

In [0]:
class AutoEncoder(nn.Module):
  def __init__(self):
    super(AutoEncoder, self).__init__()
    self.encoder = nn.Sequential(
    nn.Linear(116, 100), # TODO
    nn.BatchNorm1d(num_features=100),
    nn.ReLU(),
    nn.Linear(100, 80),
    nn.ReLU(),
    )
    self.decoder = nn.Sequential(
    nn.Linear(80, 100),# TODO
    nn.ReLU(),
    nn.Linear(100, 116),# TODO
    nn.ReLU()
    )
    self.sig = nn.Sigmoid()
  def forward(self, x):
    x = self.encoder(x)
    x = self.decoder(x)
    x[:,24:] = self.sig(x[:,24:])
    return x

In [0]:
def get_accuracy(model, data_loader):
    """Return the "accuracy" of the autoencoder model across a data set
    
    Args:
       - model: the autoencoder model, an instance of nn.Module
       - data_loader: an instance of torch.utils.data.DataLoader

    Example (to illustrate how get_accuracy is intended to be called.
             depending on your variable naming this code might not work
             out of the box)

        >>> model = AutoEncoder()
        >>> vdl = torch.utils.data.DataLoader(data_valid, batch_size=256, shuffle=True)
        >>> get_accuracy(model, vdl)
    """
    total = 0
    acc = 0
    count = 0
    for j in range(24):
        for item in data_loader: # minibatches
            inp = item.detach().numpy()
            out = model(zero_out_feature(item.clone())).detach().numpy()
            for i in range(out.shape[0]): # record in minibatch
                acc += np.square(int(out[i][j]) - inp[i][j] )
                count += 1
    return np.sqrt(acc)/count

In [0]:
import matplotlib.pyplot as plt
def plot_training_curve(path):
    train_err = np.loadtxt("{}_train_acc.csv".format(path))
    val_err = np.loadtxt("{}_val_acc.csv".format(path))
    train_loss = np.loadtxt("{}_train_loss.csv".format(path))
    val_loss = np.loadtxt("{}_val_loss.csv".format(path))
    plt.title("Train vs Validation Accuracy")
    plt.plot(range(1,train_err.shape[0]+1), train_err, label="Train")
    plt.plot(range(1,val_err.shape[0]+1), val_err, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(loc='best')
    plt.show()
    plt.title("Train vs Validation Loss")
    plt.plot(range(1,train_loss.shape[0]+1), train_loss, label="Train")
    plt.plot(range(1,val_loss.shape[0]+1), val_loss, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.show()
def zero_out_feature(records):
    """ Set the feature missing in records, by setting the appropriate
    columns of records to 0
    """
    col = random.randint(0, 23)
    records[:, col:24] = 0
    return records

def zero_out_random_feature(records):
    """ Set one random feature missing in records, by setting the 
    appropriate columns of records to 0
    """
    return zero_out_feature(records)
def validation_eval(model, valid_loader, criterion):
    valid_error = 0
    total_valid_loss = 0
    for data_val in valid_loader:
        recon=model(data_val)
        loss = criterion(recon, data_val)
        total_valid_loss += loss.item()
    return total_valid_loss/len(valid_loader),get_accuracy(model, valid_loader)
        
def train(model, train_loader, valid_loader, num_epochs=5, learning_rate=1e-4, batch_size=1,betas=(0.9, 0.999)):
    """ Training loop. You should update this."""
    torch.manual_seed(42)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,betas=betas)
    train_err = []    
    train_loss = []  
    val_err = []  
    val_loss = []
    for epoch in range(num_epochs):
        train_error = 0
        total_train_loss = 0
        train_idx = 0
        for data in train_loader:
            data_zero = zero_out_feature(data.clone()) # zero out one categorical feature
            recon = model(data_zero)
            loss = criterion(recon, data)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            total_train_loss += loss.item()
            if train_idx%1000==0:
                train_err.append(get_accuracy(model,  train_loader))
                train_loss.append(total_train_loss/(train_idx+1))
                valid_loss,valid_err = validation_eval(model, valid_loader,criterion)
                val_loss.append(valid_loss)
                val_err.append(valid_err)
            train_idx+=1
        train_loss.append(total_train_loss/len(train_loader))
        valid_loss,_= validation_eval(model, valid_loader,criterion)
        val_loss.append(valid_loss)
        print(("Epoch {}: Train loss: {} Train flow loss: {} |"+               
               "Validation loss: {} Valid flow loss: {}").format(
            epoch + 1, train_loss[-1],train_err[-1],val_loss[-1],val_err[-1]))
        model_path = "Encoder_Decoder_Batchsize{}_lr{}_epoch{}_betas{}".format(batch_size,learning_rate,epoch,betas)
        torch.save(model.state_dict(), model_path)
    print('Finished Training')    
    np.savetxt("{}_train_loss.csv".format(model_path), np.array(train_loss))       
    np.savetxt("{}_val_loss.csv".format(model_path), np.array(val_loss))
    return model_path


In [62]:
Encoder=AutoEncoder()
batch_size=32
train_loader =torch.utils.data.DataLoader(train_data,batch_size=batch_size) 
valid_loader =torch.utils.data.DataLoader(validation_data,batch_size=batch_size) 
test_loader =torch.utils.data.DataLoader(test_data,batch_size=batch_size) 
path = train(Encoder, train_loader, valid_loader, num_epochs=30, learning_rate=1e-2, batch_size=batch_size)

Epoch 1: Train loss: 69306.07981503282 Train flow loss: 1.2681110222220153 |Validation loss: 58525.716519835994 Valid flow loss: 2.7753524469239537
Epoch 2: Train loss: 60997.15062844844 Train flow loss: 0.8089677564642701 |Validation loss: 48512.25687056738 Valid flow loss: 1.7382892540594226
Epoch 3: Train loss: 50883.528160524875 Train flow loss: 0.7394963926787577 |Validation loss: 30749.360074523494 Valid flow loss: 1.5833720854040476
Epoch 4: Train loss: 40494.286465393714 Train flow loss: 0.6398757666825677 |Validation loss: 28172.23193359375 Valid flow loss: 1.3622215159140114
Epoch 5: Train loss: 38639.433544327316 Train flow loss: 0.6119368734979769 |Validation loss: 24366.435335632756 Valid flow loss: 1.3044476213957465
Epoch 6: Train loss: 37482.649598747266 Train flow loss: 0.5986780478014666 |Validation loss: 26166.26058808455 Valid flow loss: 1.258365554018844
Epoch 7: Train loss: 35533.15435633086 Train flow loss: 0.5930652197053082 |Validation loss: 24701.67039110982 V